In [4]:
import json
import pydeck as pdk
import pandas as pd

# 서울시의 경계를 나타내는 GeoJSON 데이터를 불러옵니다.
with open("seoul_boundary.geojson", "r", encoding="utf-8") as f:
    seoul_boundary = json.load(f)

# 서울시 경계를 표시하는 레이어를 생성합니다.
boundary_layer = pdk.Layer(
    "GeoJsonLayer",
    seoul_boundary, 
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation=1000,  # 높이 설정
    get_fill_color=[180, 180, 180],  # 색상 설정
    get_line_color=[255, 255, 255],
)

# 뷰 설정
view_state = pdk.ViewState(
    latitude=37.5665,  # 서울시 중심 위도
    longitude=126.9780,  # 서울시 중심 경도
    zoom=10,
    pitch=50,
)

def visualize_population(time, day):
    # 파일명 생성
    filename = f"rrrgu{time}_{day}.csv"

    # 데이터 로딩
    df = pd.read_csv(filename, encoding="utf-8")

    # 라인 데이터 생성
    line_data = df.apply(
        lambda row: {"source": [row["y1"], row["x1"]],
                     "target": [row["y2"], row["x2"]]}, axis=1).tolist()

    # 레이어 설정
    population_layer = pdk.Layer(
        "ArcLayer",
        line_data,
        get_source_position="source",
        get_target_position="target",
        get_width=1,
        get_source_color=[255, 0, 0, 80],
        get_target_color=[0, 255, 0, 80],
        pickable=True,
        auto_highlight=True,
    )

    # 렌더링
    r  = pdk.Deck(layers=[boundary_layer, population_layer], initial_view_state=view_state)
    r.to_html(f'arc_layer_{time}_{day}.html')

# 주중과 주말에 대해 0시부터 23시까지 데이터 시각화
for time in range(24):
    visualize_population(time, "주중")
    visualize_population(time, "주말")


In [7]:
import json
import pydeck as pdk
import pandas as pd

# 서울시의 경계를 나타내는 GeoJSON 데이터를 불러옵니다.
with open("seoul_boundary.geojson", "r", encoding="utf-8") as f:
    seoul_boundary = json.load(f)

# 뷰 설정
view_state = pdk.ViewState(
    latitude=37.5665,  # 서울시 중심 위도
    longitude=126.9780,  # 서울시 중심 경도
    zoom=10,
    pitch=50,
)

def visualize_population(time, day):
    # 파일명 생성
    filename = f"rrrgu{time}_{day}.csv"

    # 데이터 로딩
    df = pd.read_csv(filename, encoding="utf-8")

    # 라인 데이터 생성
    line_data = df.apply(
        lambda row: {"source": [row["y1"], row["x1"]],
                     "target": [row["y2"], row["x2"]]}, axis=1).tolist()

    # 각 지역에 대한 통계 데이터 생성
    stats_data = {}
    for data in line_data:
        source = tuple(data["source"])
        target = tuple(data["target"])
        if source not in stats_data:
            stats_data[source] = 0
        if target not in stats_data:
            stats_data[target] = 0
        stats_data[source] -= 1
        stats_data[target] += 1

    # GeoJSON 데이터에 통계 데이터 추가
    for feature in seoul_boundary["features"]:
        coordinates = tuple(feature["geometry"]["coordinates"][0][0])
        if coordinates in stats_data:
            feature["properties"]["value"] = stats_data[coordinates]
        else:
            feature["properties"]["value"] = 0



    # 단계구분도 레이어 생성
    choropleth_layer = pdk.Layer(
        "GeoJsonLayer",
        seoul_boundary,
        opacity=0.8,
        stroked=True,
        filled=True,
        extruded=False,
        wireframe=False,
        get_fill_color="[value * 100, 255, value * 100]",
        get_line_color=[255, 255, 255],
    )

    # 렌더링
    r = pdk.Deck(layers=[choropleth_layer], initial_view_state=view_state)
    r.to_html(f'choropleth_layer_{time}_{day}.html')

# 주중과 주말에 대해 0시부터 23시까지 데이터 시각화
for time in range(24):
    visualize_population(time, "주중")
    visualize_population(time, "주말")


TypeError: unhashable type: 'list'